# TP 4 RNNs: classification de séquences, forecasting, génération de séquences (suite)

_Ismaël Bonneau_

Ce notebook sert uniquement à présenter nos résultats, et les bouts de code intéressants dans le cadre de ce rapport. L'intégralité du code est contenu dans le fichier tp4.py.

## But

Nous avons dans le notebook précédent codé un RNN from scratch et nous l'avons appliqué sur une tâche de classification de séquence. Nous allons maintenant reprendre ce RNN et réaliser une tâche de génération de séquence.

## Données

Le jeu de données suggéré dans l'énoncé est un discours de Donald Trump. Cependant, il est beaucoup plus drôle de travailler sur le script du seigneur des anneaux. 

<img src="boromir.jpg">

In [1]:
import numpy as np
import torch
import torch.nn as nn

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

%matplotlib inline
%load_ext autoreload
%autoreload 2

import models
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [33]:
bestmovieever = pd.read_csv("data/lord-of-the-rings-data/lotr_scripts.csv")

### Récupérons les lignes de dialogue des 3 films du seigneur des anneaux. Nous allons garder uniquement les répliques de Gandalf.

In [35]:
bestmovieever[bestmovieever["char"] == "GANDALF"].head()

,Unnamed: 0,char,dialog,movie
34,34,GANDALF,Now come the days of the King. May they be...,The Return of the King
68,68,GANDALF,Hobbits!,The Return of the King
72,72,GANDALF,"Be careful. Even in defeat, Saruman is dangero...",The Return of the King
74,74,GANDALF,"No, we need him alive. We need him to talk.",The Return of the King
78,78,GANDALF,Your treachery has already cost many lives. Th...,The Return of the King


### On nettoie un peu les lignes de dialogue:

In [57]:
from gensim.parsing.preprocessing import preprocess_string
from gensim.parsing.preprocessing import strip_multiple_whitespaces, strip_tags

CUSTOM_FILTERS = [strip_tags, strip_multiple_whitespaces]

hey = list(bestmovieever[bestmovieever["char"] == "GANDALF"]["dialog"])
gandalf = []
for d in hey:
    gandalf.append(" ".join(preprocess_string(d, CUSTOM_FILTERS)))

In [59]:
gandalf[0]

'Now come the days of the King. May they be blessed.'

### Nous allons travailler au niveau des caractères. 

il faut encoder chaque caractère sous forme d'id numérique (entier).

In [68]:
import string
import unicodedata

LETTRES = string.ascii_letters + string.punctuation + string.digits + ' '
id2lettre = dict(zip(range(1, len(LETTRES)+1),LETTRES))
id2lettre[0]= ' ' ##NULL CHARACTER
lettre2id = dict(zip(id2lettre.values(), id2lettre.keys()))

def normalize(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if c in LETTRES)
def string2code(s):
    return torch.tensor([lettre2id [c] for c in normalize(s)])
def code2string(t):
    if type(t) != list:
        t = t.tolist()
    return ''.join(id2lettre[i] for i in t)

In [69]:
string2code(gandalf[0])

tensor([40, 15, 23,  0,  3, 15, 13,  5,  0, 20,  8,  5,  0,  4,  1, 25, 19,  0,
        15,  6,  0, 20,  8,  5,  0, 37,  9, 14,  7, 66,  0, 39,  1, 25,  0, 20,
         8,  5, 25,  0,  2,  5,  0,  2, 12,  5, 19, 19,  5,  4, 66])